# Implement Continious Bag of Words (CBOW) Model

# Imports

In [1]:
import warnings
warnings.filterwarnings("ignore",category=UserWarning)

In [3]:
import numpy as np
import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense, Embedding, Lambda
from keras.optimizers import SGD
from keras.utils import np_utils
from sklearn.metrics.pairwise import euclidean_distances

# Load Data

In [4]:
file1 = open("Metamorphosis.txt","r")

In [5]:
data = file1.read()

In [6]:
data

'One morning as Gregor Samsa was waking up from his anxious dreams, he discovered that in his bed, he had been changed into a monsterous verminous bug. '

In [7]:
words = data.split()

In [8]:
words = [x.lower().replace(",","").replace(".","") for x in words]

In [9]:
words

['one',
 'morning',
 'as',
 'gregor',
 'samsa',
 'was',
 'waking',
 'up',
 'from',
 'his',
 'anxious',
 'dreams',
 'he',
 'discovered',
 'that',
 'in',
 'his',
 'bed',
 'he',
 'had',
 'been',
 'changed',
 'into',
 'a',
 'monsterous',
 'verminous',
 'bug']

In [10]:
vocab = set(words)

In [11]:
vocab_size = len(vocab)

In [12]:
vocab_size

25

In [13]:
word_to_idx = {word: i for i, word in enumerate(vocab)}
idx_to_word = {i: word for i, word in enumerate(vocab)}

In [14]:
word_to_idx

{'bug': 0,
 'his': 1,
 'anxious': 2,
 'verminous': 3,
 'one': 4,
 'waking': 5,
 'morning': 6,
 'he': 7,
 'was': 8,
 'been': 9,
 'bed': 10,
 'samsa': 11,
 'monsterous': 12,
 'discovered': 13,
 'a': 14,
 'up': 15,
 'in': 16,
 'that': 17,
 'as': 18,
 'changed': 19,
 'had': 20,
 'into': 21,
 'from': 22,
 'dreams': 23,
 'gregor': 24}

In [15]:
idx_to_word

{0: 'bug',
 1: 'his',
 2: 'anxious',
 3: 'verminous',
 4: 'one',
 5: 'waking',
 6: 'morning',
 7: 'he',
 8: 'was',
 9: 'been',
 10: 'bed',
 11: 'samsa',
 12: 'monsterous',
 13: 'discovered',
 14: 'a',
 15: 'up',
 16: 'in',
 17: 'that',
 18: 'as',
 19: 'changed',
 20: 'had',
 21: 'into',
 22: 'from',
 23: 'dreams',
 24: 'gregor'}

# Generate Context-Word Pairs

In [16]:
c = 2
dim = 100

In [17]:
X_train = []
Y_train = []

for index, i in enumerate(words):
    
    #edge case
    if index <= 1 or index >= len(words)-2:
        continue
        
    # sliding context window 
    start = index - c
    end = index + c + 1
    # 2 words Left and 2 words Right
    context = words[start:end]
    context.remove(i)
    target = i
    
    # data
    X_train.append([word_to_idx[w] for w in context])
    Y_train.append([word_to_idx[target]])

In [18]:
X_train = np.array(X_train)
Y_train = np.array(Y_train)

In [19]:
Y_train = np_utils.to_categorical(Y_train, vocab_size)

# Build Model

In [21]:
cbow = Sequential()

In [22]:
cbow.add(Embedding(input_dim=vocab_size, output_dim=dim, input_length= c*2))
cbow.add(Lambda(lambda x: K.mean(x, axis=1), output_shape=(dim,)))
cbow.add(Dense(vocab_size, activation='softmax'))

In [23]:
cbow.compile(loss="categorical_crossentropy", optimizer=SGD(), metrics=["accuracy"])

In [24]:
cbow.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 4, 100)            2500      
                                                                 
 lambda (Lambda)             (None, 100)               0         
                                                                 
 dense (Dense)               (None, 25)                2525      
                                                                 
Total params: 5,025
Trainable params: 5,025
Non-trainable params: 0
_________________________________________________________________


In [25]:
cbow.fit(X_train, Y_train, epochs=5000, batch_size=256)

Epoch 1/5000
1/1 [==============================] - 0s 312ms/step - loss: 3.2187 - accuracy: 0.0000e+00
Epoch 2/5000
1/1 [==============================] - 0s 3ms/step - loss: 3.2185 - accuracy: 0.0000e+00
Epoch 3/5000
1/1 [==============================] - 0s 4ms/step - loss: 3.2182 - accuracy: 0.0435
Epoch 4/5000
1/1 [==============================] - 0s 5ms/step - loss: 3.2179 - accuracy: 0.0435
Epoch 5/5000
1/1 [==============================] - 0s 4ms/step - loss: 3.2177 - accuracy: 0.0435
Epoch 6/5000
1/1 [==============================] - 0s 4ms/step - loss: 3.2174 - accuracy: 0.0435
Epoch 7/5000
1/1 [==============================] - 0s 5ms/step - loss: 3.2171 - accuracy: 0.0435
Epoch 8/5000
1/1 [==============================] - 0s 4ms/step - loss: 3.2169 - accuracy: 0.0870
Epoch 9/5000
1/1 [==============================] - 0s 4ms/step - loss: 3.2166 - accuracy: 0.0870
Epoch 10/5000
1/1 [==============================] - 0s 4ms/step - loss: 3.2164 - accuracy: 0.0870
Epoch 11/

1/1 [==============================] - 0s 4ms/step - loss: 3.1974 - accuracy: 0.2609
Epoch 84/5000
1/1 [==============================] - 0s 4ms/step - loss: 3.1971 - accuracy: 0.2609
Epoch 85/5000
1/1 [==============================] - 0s 4ms/step - loss: 3.1969 - accuracy: 0.2609
Epoch 86/5000
1/1 [==============================] - 0s 3ms/step - loss: 3.1966 - accuracy: 0.2609
Epoch 87/5000
1/1 [==============================] - 0s 3ms/step - loss: 3.1964 - accuracy: 0.2609
Epoch 88/5000
1/1 [==============================] - 0s 4ms/step - loss: 3.1961 - accuracy: 0.2609
Epoch 89/5000
1/1 [==============================] - 0s 3ms/step - loss: 3.1958 - accuracy: 0.2609
Epoch 90/5000
1/1 [==============================] - 0s 3ms/step - loss: 3.1956 - accuracy: 0.2609
Epoch 91/5000
1/1 [==============================] - 0s 4ms/step - loss: 3.1953 - accuracy: 0.2609
Epoch 92/5000
1/1 [==============================] - 0s 4ms/step - loss: 3.1951 - accuracy: 0.2609
Epoch 93/5000
1/1 [=====

1/1 [==============================] - 0s 4ms/step - loss: 3.1767 - accuracy: 0.1739
Epoch 166/5000
1/1 [==============================] - 0s 4ms/step - loss: 3.1765 - accuracy: 0.1739
Epoch 167/5000
1/1 [==============================] - 0s 3ms/step - loss: 3.1762 - accuracy: 0.1739
Epoch 168/5000
1/1 [==============================] - 0s 3ms/step - loss: 3.1760 - accuracy: 0.1739
Epoch 169/5000
1/1 [==============================] - 0s 4ms/step - loss: 3.1757 - accuracy: 0.1739
Epoch 170/5000
1/1 [==============================] - 0s 3ms/step - loss: 3.1755 - accuracy: 0.1739
Epoch 171/5000
1/1 [==============================] - 0s 4ms/step - loss: 3.1752 - accuracy: 0.1739
Epoch 172/5000
1/1 [==============================] - 0s 3ms/step - loss: 3.1750 - accuracy: 0.1739
Epoch 173/5000
1/1 [==============================] - 0s 3ms/step - loss: 3.1747 - accuracy: 0.1739
Epoch 174/5000
1/1 [==============================] - 0s 3ms/step - loss: 3.1745 - accuracy: 0.1739
Epoch 175/5000


1/1 [==============================] - 0s 4ms/step - loss: 3.1566 - accuracy: 0.1739
Epoch 248/5000
1/1 [==============================] - 0s 5ms/step - loss: 3.1564 - accuracy: 0.1739
Epoch 249/5000
1/1 [==============================] - 0s 3ms/step - loss: 3.1561 - accuracy: 0.1739
Epoch 250/5000
1/1 [==============================] - 0s 5ms/step - loss: 3.1559 - accuracy: 0.1739
Epoch 251/5000
1/1 [==============================] - 0s 4ms/step - loss: 3.1556 - accuracy: 0.1739
Epoch 252/5000
1/1 [==============================] - 0s 4ms/step - loss: 3.1554 - accuracy: 0.1739
Epoch 253/5000
1/1 [==============================] - 0s 5ms/step - loss: 3.1552 - accuracy: 0.1739
Epoch 254/5000
1/1 [==============================] - 0s 3ms/step - loss: 3.1549 - accuracy: 0.1739
Epoch 255/5000
1/1 [==============================] - 0s 5ms/step - loss: 3.1547 - accuracy: 0.1739
Epoch 256/5000
1/1 [==============================] - 0s 4ms/step - loss: 3.1544 - accuracy: 0.1739
Epoch 257/5000


1/1 [==============================] - 0s 3ms/step - loss: 3.1370 - accuracy: 0.1739
Epoch 330/5000
1/1 [==============================] - 0s 4ms/step - loss: 3.1368 - accuracy: 0.1739
Epoch 331/5000
1/1 [==============================] - 0s 3ms/step - loss: 3.1366 - accuracy: 0.1739
Epoch 332/5000
1/1 [==============================] - 0s 3ms/step - loss: 3.1363 - accuracy: 0.1739
Epoch 333/5000
1/1 [==============================] - 0s 4ms/step - loss: 3.1361 - accuracy: 0.1739
Epoch 334/5000
1/1 [==============================] - 0s 80ms/step - loss: 3.1359 - accuracy: 0.1739
Epoch 335/5000
1/1 [==============================] - 0s 3ms/step - loss: 3.1356 - accuracy: 0.1739
Epoch 336/5000
1/1 [==============================] - 0s 3ms/step - loss: 3.1354 - accuracy: 0.1739
Epoch 337/5000
1/1 [==============================] - 0s 3ms/step - loss: 3.1352 - accuracy: 0.1739
Epoch 338/5000
1/1 [==============================] - 0s 3ms/step - loss: 3.1349 - accuracy: 0.1739
Epoch 339/5000

...

1/1 [==============================] - 0s 4ms/step - loss: 1.8264 - accuracy: 0.8261
Epoch 4631/5000
1/1 [==============================] - 0s 5ms/step - loss: 1.8260 - accuracy: 0.8261
Epoch 4632/5000
1/1 [==============================] - 0s 4ms/step - loss: 1.8255 - accuracy: 0.8261
Epoch 4633/5000
1/1 [==============================] - 0s 4ms/step - loss: 1.8251 - accuracy: 0.8261
Epoch 4634/5000
1/1 [==============================] - 0s 4ms/step - loss: 1.8247 - accuracy: 0.8261
Epoch 4635/5000
1/1 [==============================] - 0s 4ms/step - loss: 1.8243 - accuracy: 0.8261
Epoch 4636/5000
1/1 [==============================] - 0s 5ms/step - loss: 1.8239 - accuracy: 0.8261
Epoch 4637/5000
1/1 [==============================] - 0s 4ms/step - loss: 1.8235 - accuracy: 0.8261
Epoch 4638/5000
1/1 [==============================] - 0s 4ms/step - loss: 1.8230 - accuracy: 0.8261
Epoch 4639/5000
1/1 [==============================] - 0s 4ms/step - loss: 1.8226 - accuracy: 0.8261
Epoch 

1/1 [==============================] - 0s 4ms/step - loss: 1.7927 - accuracy: 0.8261
Epoch 4712/5000
1/1 [==============================] - 0s 4ms/step - loss: 1.7923 - accuracy: 0.8261
Epoch 4713/5000
1/1 [==============================] - 0s 5ms/step - loss: 1.7919 - accuracy: 0.8261
Epoch 4714/5000
1/1 [==============================] - 0s 4ms/step - loss: 1.7915 - accuracy: 0.8261
Epoch 4715/5000
1/1 [==============================] - 0s 5ms/step - loss: 1.7911 - accuracy: 0.8261
Epoch 4716/5000
1/1 [==============================] - 0s 4ms/step - loss: 1.7907 - accuracy: 0.8261
Epoch 4717/5000
1/1 [==============================] - 0s 4ms/step - loss: 1.7903 - accuracy: 0.8261
Epoch 4718/5000
1/1 [==============================] - 0s 5ms/step - loss: 1.7898 - accuracy: 0.8261
Epoch 4719/5000
1/1 [==============================] - 0s 4ms/step - loss: 1.7894 - accuracy: 0.8261
Epoch 4720/5000
1/1 [==============================] - 0s 5ms/step - loss: 1.7890 - accuracy: 0.8261
Epoch 

1/1 [==============================] - 0s 3ms/step - loss: 1.7592 - accuracy: 0.8696
Epoch 4793/5000
1/1 [==============================] - 0s 4ms/step - loss: 1.7588 - accuracy: 0.8696
Epoch 4794/5000
1/1 [==============================] - 0s 3ms/step - loss: 1.7584 - accuracy: 0.8696
Epoch 4795/5000
1/1 [==============================] - 0s 4ms/step - loss: 1.7580 - accuracy: 0.8696
Epoch 4796/5000
1/1 [==============================] - 0s 3ms/step - loss: 1.7576 - accuracy: 0.8696
Epoch 4797/5000
1/1 [==============================] - 0s 4ms/step - loss: 1.7571 - accuracy: 0.8696
Epoch 4798/5000
1/1 [==============================] - 0s 3ms/step - loss: 1.7567 - accuracy: 0.8696
Epoch 4799/5000
1/1 [==============================] - 0s 4ms/step - loss: 1.7563 - accuracy: 0.8696
Epoch 4800/5000
1/1 [==============================] - 0s 5ms/step - loss: 1.7559 - accuracy: 0.8696
Epoch 4801/5000
1/1 [==============================] - 0s 4ms/step - loss: 1.7555 - accuracy: 0.8696
Epoch 

1/1 [==============================] - 0s 3ms/step - loss: 1.7258 - accuracy: 0.8696
Epoch 4874/5000
1/1 [==============================] - 0s 5ms/step - loss: 1.7254 - accuracy: 0.8696
Epoch 4875/5000
1/1 [==============================] - 0s 5ms/step - loss: 1.7250 - accuracy: 0.8696
Epoch 4876/5000
1/1 [==============================] - 0s 4ms/step - loss: 1.7246 - accuracy: 0.8696
Epoch 4877/5000
1/1 [==============================] - 0s 4ms/step - loss: 1.7242 - accuracy: 0.8696
Epoch 4878/5000
1/1 [==============================] - 0s 4ms/step - loss: 1.7238 - accuracy: 0.8696
Epoch 4879/5000
1/1 [==============================] - 0s 5ms/step - loss: 1.7233 - accuracy: 0.8696
Epoch 4880/5000
1/1 [==============================] - 0s 4ms/step - loss: 1.7229 - accuracy: 0.8696
Epoch 4881/5000
1/1 [==============================] - 0s 5ms/step - loss: 1.7225 - accuracy: 0.8696
Epoch 4882/5000
1/1 [==============================] - 0s 4ms/step - loss: 1.7221 - accuracy: 0.8696
Epoch 

1/1 [==============================] - 0s 4ms/step - loss: 1.6926 - accuracy: 0.8696
Epoch 4955/5000
1/1 [==============================] - 0s 5ms/step - loss: 1.6922 - accuracy: 0.8696
Epoch 4956/5000
1/1 [==============================] - 0s 4ms/step - loss: 1.6918 - accuracy: 0.8696
Epoch 4957/5000
1/1 [==============================] - 0s 3ms/step - loss: 1.6914 - accuracy: 0.8696
Epoch 4958/5000
1/1 [==============================] - 0s 3ms/step - loss: 1.6909 - accuracy: 0.8696
Epoch 4959/5000
1/1 [==============================] - 0s 4ms/step - loss: 1.6905 - accuracy: 0.8696
Epoch 4960/5000
1/1 [==============================] - 0s 4ms/step - loss: 1.6901 - accuracy: 0.8696
Epoch 4961/5000
1/1 [==============================] - 0s 4ms/step - loss: 1.6897 - accuracy: 0.8696
Epoch 4962/5000
1/1 [==============================] - 0s 3ms/step - loss: 1.6893 - accuracy: 0.8696
Epoch 4963/5000
1/1 [==============================] - 0s 5ms/step - loss: 1.6889 - accuracy: 0.8696
Epoch 

In [26]:
pred = cbow.predict(X_train)

1/1 [==============================] - 0s 59ms/step


In [27]:
pred.shape

(23, 25)

In [28]:
pred = pred.argmax(axis=1)

In [29]:
i = 0
for context in X_train:
    print("...............................")
    print("===========Context :===========")
    for w in context:
        print(idx_to_word[w])
    print("=========Predicted Target :====")
    print(idx_to_word[pred[i]])
    i+=1

...............................
===========Context :===========
one
morning
gregor
samsa
=========Predicted Target :====
as
...............................
===========Context :===========
morning
as
samsa
was
=========Predicted Target :====
gregor
...............................
===========Context :===========
as
gregor
was
waking
=========Predicted Target :====
samsa
...............................
===========Context :===========
gregor
samsa
waking
up
=========Predicted Target :====
was
...............................
===========Context :===========
samsa
was
up
from
=========Predicted Target :====
waking
...............................
===========Context :===========
was
waking
from
his
=========Predicted Target :====
up
...............................
===========Context :===========
waking
up
his
anxious
=========Predicted Target :====
from
...............................
===========Context :===========
up
from
anxious
dreams
=========Predicted Target :====
his
....................